# **Loading Amber into the TACC Machines**


Amber is a molecular simulation libary that allows a scientists to simulate various organic molecules including proteins, lipids, DNA, and RNA.

1. SSH into a TACC machine with the amber installed in it
2. Run `module spider amber` to see what other modules you need to load and load those modules (It’s `module load intel/17.0.4` for Amber16 on stampede)
4. Finally load the amber module with `module load amber`


| Machine   | Amber Version| GPU's Available |
|-----------|--------------|-----------------|
| Longhorn  |    20.0      |    &#x2612;     |
| Stampede2 |    16.0      |    &#2612;      |




GPU Features **NOT** Supported as of GPU v16.0.0

---------------------------------------------------

* ibelly /= 0 Simulations using belly style constraints are not supported.
* icfe /= 0	Support for TI is not currently implemented. An update is planned within the next few months that will add this support.
* if (igb/=0 & cut < systemsize) GPU accelerated implicit solvent GB simulations do not support a cutoff.
* nmropt > 1	Support is not currently available for nmropt > 1. In addition for nmropt = 1 only features that do not change the underlying force field parameters are supported. For example umbrella sampling restraints are supported as well as simulated annealing functions such as variation of Temp0 with simulation step. However, varying the VDW parameters with step is NOT supported.
* nrespa /= 1	No multiple time stepping is supported.
* vlimit /= -1	For performance reasons the vlimit function is not implemented on GPUs.
* es_cutoff /= vdw_cutoff	Independent cutoffs for electrostatics and van der Waals are not supported on GPUs.
* order > 4	PME interpolation orders of greater than 4 are not supported at present.
* imin=1 (in parallel) Minimization is only supported in the serial GPU code.
* netfrc is ignored	The GPU code does not currently support the netfrc correction in PME calculations and the value of netfrc in the ewald namelist is ignored.
* emil_do_calc /= 0	Emil is not supported on GPUs.
* lj264 /= 0 The 12-6-4 potential is not supported on GPUs.
* isgld > 0	Self guided langevin is not supported on GPUs.
* iemap > 0	EMAP restraints are not supported on GPUs.


# Basics

## Force Fields

**Functional form**:

The functional form of the Amber force field (neglecting implicit solvent or polarisation terms) uses the following Hamiltonian:

$$
\begin{align}
E_{\text{total}} &= \sum _{\text{bonds}} k_b(r-r_0)^2\\
&+\sum _{\text{angles}} k_{\theta}(\theta-\theta _0)^2\\
&+\sum _{\text{dihedrals}} V_n[1+\cos(n \phi-\gamma)]\\
&+\sum _{i=1}^{N-1}\sum _{j = i + 1}^N \left[\frac{A_{ij}}{R_{ij}^{12}}-\frac{B_{ij}}{R_{ij}^6}+\frac{q_i q_j}{\epsilon R_{ij}} \right]
\end{align}
$$

<font size = "1"> does not use the (physically more correct) $\frac{k_b}{2}$, $\frac{k_θ}{2}$, and $\frac{V_n}{2}$ notations because it refers to the constants as they appear in the actual parameter files. </font>



* First term (summing over bonds): represents the energy between covalently bonded atoms. This harmonic (ideal spring) force is a good approximation near the equilibrium bond length, but becomes increasingly poor as atoms separate.
* Second term (summing over angles): represents the energy due to the geometry of electron orbitals involved in covalent bonding.
* Third term (summing over torsions): represents the energy for twisting a bond due to bond order (e.g., double bonds) and neighboring bonds or lone pairs of electrons. One bond may have more than one of these terms, such that the total torsional energy is expressed as a Fourier series.
* Fourth term (double summation over $i$ and $j$): represents the non-bonded energy between all atom pairs, which can be decomposed into van der Waals (first term of summation) and electrostatic (second term of summation) energies.

<br>

**Types of force fields:**

* a protein force field (recommended choice is ff14SB)
* a DNA force field (recommended choice is OL15)
* an RNA force field (recommended choice is OL3)
* a carbohydrate force field (recommended choice is GLYCAM_06j)
* a lipid force field (recommended choice is lipid17)
* a water model with associated atomic ions (more variable, but the most common choice is still tip3p); other popular choices are spc/e, tip4pew, and OPC. Not needed if you are using an implicit solvent model.
* a general force field, for organic molecules like ligands (recommended choice is gaff2)
* other components (such as modified amino acids or nucleotides, other ions), as needed

# Obtaining a PDB file or EM Map

The [Protein Data Bank](https://www.rcsb.org/) (PDB) contains a vast library of proteins whose structures have already been solved with experimental methods such as X-ray Crystallography or NMR Spectroscopy.

EM Maps can be found at [Electron Microscopy Data Bank](https://www.ebi.ac.uk/pdbe/emdb/).

In order to find the structures/EM Maps:

1. Go to the PDB or EMDB website and search for the name of the protein
2. Once you have seen your desired protein, click "Download Files" and select the format you want (usually the .pdb for PDB files or .map/.mrc for EM maps)

If you want to do all the querying and downloading without having to leave the terminal, you can also obtain the PDB structures with FTP (File Transfer Protocol) through [PDB's FTP Server](http://ftp.rcsb.org/). `ftp ftp.rcsb.org`




If you wanted, you could also query for desired PDB ids with PDB's [REST API](https://www.rcsb.org/pdb/software/rest.do).

# Cleaning the PDB file

Cleaning the PDB file:

The pdb file must be first cleaned before amber can process it. In general, you can just use the `pdb4amber` amber tool on the pdb file to clean it. The final PDB file should only contain the locations for the heavy atoms of amino acid.

In case pdf4amber does not work completely, the actual PDB preprocessing requirements are in the [Amber Reference Manuals](https://ambermd.org/Manuals.php).  

Some basic requirements are listed below:

* Save ligands in MDL standard data files (SDF) or in PDB format and convert it to a SDF file later. The SDF file should include correct bond order and all hydrogens. In addition, the ligand’s heavy atom coordinates should be kept at their original location.

* Delete irrelevant water molecules. If the water is needed for ligand binding, they must be specified as residues by the names WAT or HOH.

* Delete irrelevant ions or co-factors

* Delete protein hydrogens that are explicitly expressed in the PDB file

* Remove all connectivity records (disulfide bonds will be reconnected later)

* Use TER records to separate parts in the PDB file that are not connected covalently. This applies to individual water molecules that need to be kept.

Running, `pdb4amber -i input.pdb -o output.pdb --dry --reduce` should delete the water molecules (hence the --dry option) and add hydrogens where necessary (hence the --reduce option).


# LEaP

<font size = "4">Overview:</font>

LEaP prepares input files for simulation programs, by collating parameters to make a complete description of a molecule. It serves as a portal between many chemical structure file types (.pdb), and the Amber model parameter file types (.lib, .prepi, parm.dat, and .frcmod).

LEaP generates an AMBER topology file (.prmtop) and coordinate file (.inpcrd).

- The coordinates reflects the structure it was given, after filling out any incomplete residues or constructing what it was told to make.

- The topology is a complete description of how that system will behave. The simulation is needed because the system is almost certainly so complex that analytic solutions are not to be found.


<br>


<font size = "4">Steps to Build a Molecule for Molecular Dynamics:</font>

1. Load UNIT or PARMSET objects.
2. Construct molecule within LEaP
3. Output topology and coordinate files from LEaP to use in AMBER.

EX:

---
```bash
source leaprc.protein.ff14SB (load a force field)
x = loadPdb trypsin.pdb (load in a structure)
.... add in cross-links, solvate, etc.
saveAmberParm x prmtop prmcrd (save files)
```
---

# Sander and Pmemd

The two main programs that actually run simulations in Amber are Sander and its MPI version Pmemd. Although there are some features that pmemd does not support but sander does (and vice versa), they share many of the same capabilities. The main difference is that pmemd is parallelized and optimized for GPUs. As the TACC supercomputers are well-equipped for parallelized tasks, **it is usually faster to run simulations with pmemd rather than sander**. The basic syntax for sander is shown below. Besides replacing `sander` at the beginning with pmemd, the syntax for pmemd is the same.

> sander [**-O**] [**-A**] **-i** md.in **-o** md.out **-p** parm.prmtop **-c** coor.inpcrd **-y** traij.inptraj **-r** rst.restart **-x** coor.nc **-v** vel.mdvel **-frc** frc.mdfrc **-e** eng.mden **-inf** info.mdinfo

Flags:

> **-O**: Overwrite all output files if they exist already

> **-A**: Append to all output files

Inputs:

> **-i**: Input Simulation file

> **-p**: Amber Parameter file (generated in tleap/xleap)

> **-c**: Amber Coordinate file containing intial coordinates, velocities, and period box size (generated in tleap/xleap)

> **-y**: Input Amber trajectory file (used when *imin=5*)

Outputs:

> **-o**: Ouput Simulation file

> **-r**: Amber restart file that contains the information required to continue the simulation or start another simulation with the last simulations coordinates, velocities, and time step

> **-x**: Ouput Amber coordinate trajectory file (can be either .nc or .mdcrd file)

> **-v**: Output Amber velocity trajectory file

> **-frc**: Output Amber force trajectory file

> **-e**: Ouput Amber energy trajectory file (**not synchronized with coordinate or velcoity trajectory files**)

> **-inf**: Latest recorded simulation energy information (only contains the energy state of the simulation when it ends)


# Simulating Simple Protein in Amber (NOT COMPLETE)

Once you have a cleaned pdb file or the sequence of the protein, we can begin to prep it for the simulation with Leap. A general procedure is provided below.

1. Open up tleap (or xleap which is the GUI version) by running `tleap`

2. For proteins, we use the recommended protein force field FF14SB. We load in force fields in amber with the `source` command.
```bash
source leaprc.protein.ff14SB
```
3. We can then either build the sequence manually with the `sequence` command as
```bash
protein = sequence {NAA1 AA2 AA3 ... CAAL}
```
Notice how we have to cap the first and last amnino acids with N and C repsectively.
or we can load a cleaned PDB file ("Cleaning the PDB" for more information) with
```bash
protein = loadpdb pdb_file.pdb
```

4. For this simulation, we will use an explicit solvent (the other option is an implicit solvent). As we will use water as the solvent, let's load in the TIP3P water solvent.
```bash
source leaprc.water.tip3p
```

5. When using an explicit solvent, Amber places the atoms in a periodic box, in which atoms that happen to pass through one end of the box appear on the opposite end. t














# Protein Folding Simulations with Cryo-EM Map Constraints (NOT COMPLETE)



In order to simulate proteins folding with Cryo-EM map constraints, we can use the `&emap` namelist in the simulation file. **Note that as of Amber 20, Amber does not support emap constraints for GPUs**.  

# Parameters for Sander/Pmemd Simulations

## General Parameters

**imin:** Defines whether we are doing minimizaton or not

>0: (default) Set when running an MD without minimization

>1: Set when performing energy minimization

>5: Set when reading in a trajectory file as input

**irest:** Defines whether we use the restart file or start a new simulation

>0: (default) Start a new simulation at time step 0. Ignore velocities in the input coordinate file. Set this flag when you do not want to continue from the restart files last time step.

>1: Read in the coordinates and velocities from the restart file and continue simulation from last time step in the restart file. Remember to set the ntx flag to 5 so that velocities can be read. (**Very useful when you run out of wall time and want to continue the simulation**)

**ntx:** Defines whether we read in the velocities from the input file  

>1: (default) Read in coordinates but not velocities from the input. Usually set when irest=0.

>5: Read both coordinates and velocities from the input. **Must set when irest=1.**

## Input/Output Format Parameters

**ntpr**: Prints *energy information* to the "mdout" and "mdinfo" files every *ntpr* time steps

> Default: 50

**ntwr**:

> *ntwr* > 0: Writes to the restart file every *ntwr* time steps

> *ntwr* < 0: Writes to a unique restart file (i.e. restrt_`nstep`) every abs(*ntwr*) time steps

> Default: nstlim (nstlim is the total number of time steps in the simulation)

**ntwx:** Writes the coordinates to the "mdcrd" file (trajectory file) every *ntwx* time steps

> Default: 0

**ntwprt**: Defines the number of atoms to include in the trajectory file ("mdcrd" and "mdvel")

> 0: (default) Include all atoms in the trajectory files

> *ntwprt* > 0: Include only atoms 1 to *ntwprt* in the trajectory files


## Minimization Parameters

**maxcyc**: Defines the maximum number of minimization cycles

> Default: 1

**ntmin**: Defines the minimization method

> 0: Full conjugate gradient minimization

> 1: (default) *ncyc* cycles of steepest descent method followed the rest of the cycles being conjugate gradient

> 2: Only steepest descent Method

> 3: XMIN method

> 4: LMOD Method

**ncyc**: Defines the number of times steepest descent is performed at the beginnging of minimzation when *ntmin*=1

> Default: 10



## Molecular Dynamics Parameters

### General MD Parameters

**nstlim**: Defines the number of MD steps to be performed

>Default: 1

**dt**: The time step in picoseconds. The maximum is 0.002 if SHAKE is used and 0.001 if it isn't. For temperatures above 300K, the step size should be reduced. Otherwise, the increased magnitude of the velocities and greater distance traveled between each force evaluation may lead to extremely high energies and system blowup.

>Default: 0.001

**nscm**: Defines the interval (in time steps) at which translational and rotational center-of-mass motion is removed.

For non-periodic simulations, translational and rotational motion is removed every *nscm* steps.

For periodic simulations, only translational motion is removed every *nscm* steps.

For Langevin dynamics, the position of the center-of-mass of the molecules is reset to zero every *nscm* steps but not the velocities.

> Default: 1000
























### Temperature Regulation Parameters

#### General Temperature Regulation Parameters

**ntt:** Defines the temperature control method. There are several types of temperature regulation methods.

NVE (or microcanonical) ensembles hold the system constant with respect to the number of atoms(N), the volume (V), and the energy (E). They approach the NVT (or canonical) ensemble as the number of degrees of freedom approaches infinity. In these systems, the temperature often varies. *ntt=0* is the only one temperature regulation scheme that is simulates an NVE ensemble.

NVT ensembles hold the system constant with respect to the number of atoms (N), the volume (V), and the temperature (T). In order to keep the temperature constant, a "thermostat" is used that regulates energy leaving and exiting the system.

NPT ensembles hold the system constant with respect to the number of atoms (N), the pressure (P), the temperature (T). They are generally not applicable to biological systems.

> 0: Constant total energy clasical dynamics (NVE ensemble). **In general, do not use this method.**

> 1: Constant temperature with the weak-coupling algorithm. This method only ensures that the total kinetic energy is approximate for the desired temperature, but it does not ensure that the temperature is even over all parts of the molecule. Atomic collisions can effect an even temperature distribution but this is not guraanteed.

> 2: [Andersen](https://aip.scitation.org/doi/10.1063/1.439486)-like coupling scheme where imaginary "collisions" randomize the velocities to distriubution correspondign to *temp0* every *vrand* steps. In between these artifical collisions, the dynamics are Newtonian (normal). Too high of a collision rate will cause the molecules to explore the configuration space slower. Too low of a collision rate will cause the distribution of energies to be sampled slower.

> 3: [Langevin dynamics](https://cmm.cit.nih.gov/intro_simulation/node24.html) with collision frequency given by *gamma_ln*. Make sure to the set the *ig* parameter is a different value at each restart of a simulation (setting it to -1 ensures this). **In general, use this method**.

>9: Optimized Isokinetic Nose-Hoover chain ensemble:

> 10: Stochastic Isokinetic Nose-Hoover RESPA integrator:

**temp0:** Defines the reference temperture in Kelvins that the system should be kept at if *ntt>0*.

> Default: 300

**tempi**: Defines the initial temperature of the system in Kelvins

> 0: (default) The velocities are calculated from the forces

> *tempi* > 0: The velocities are sampled from a Maxwellian distribution at *tempi* Kelvin.

**ig**: The seed for the MD simulations random number generator. This value affects the initial velcoities when *tempi* $\neq$ 0 and *ntx* = 1. It also affect the initalizations of ithe Langevin dynamics and Andersen coupling (*ntt* = 2 or 3) and should be set to different values every restart.

> -1: (default) the random seed is the current data and time

**vlimit**: Reduces any velocity greater than abs(*vlimit*) to *vlimit* (preserving the sign). This used to prevent occasional instabilities.

> Default: 20

> 0: No limit




#### Weak-Coupling Algorithm Parameters (ntt=1)

**tautp**: Time constant in picoseconds for heat bath coupling. The value should be in between 0.5 and 5.0 ps, Smaller values provide tigher coupling to the heat bath and thus a faster heating, smaller fluctuations in kinetic energy, larger fluctuations in total energy, and a less natural trajectory. Values much larger than the length of the simulation result in constant energy.

> Default: 1.0

Andersen-Coupling Parameters (ntt=2)

**vrand**: The velocities are randomized to *temp0* every *vrand* steps.

> Default: 1000

#### Langevin Dynamics and Nose-Hoover Parameters (ntt=3)

**gamma_ln**:

When *ntt=3*, this value defines the collision frequence $\gamma$ in $\text{ps}^{-1}$. A Leapfrog integrator is used to adjust the dynamic. When selecting *gamma_ln*, we do not need to use the physical collision frequencies (e.g 50 $\text{ps}^{-1}$ for water). For stability of integration and sampling, **we often use values much smaller in the  range of 2 to 5 $\text{ps}^{-1}$. For implicit solvents (GB), we use even lower values such as 0.01 $\text{ps}^{-1}$ to speed up the simulation,. sometimes 100 fold.**

When *ntt=9*, this value defines the thermostat coupling constant  for the Optimized Isokinetic Nose-Hoover cahin integrator, which is equivalent to *1/gamma_ln*.

When *ntt=10*, this value is the friction constant associated with the stochastic component of the integrator.

> Default: 0

### Self-Guided Langevin Dynamics Parameters

**isgld**: Set when performing self-guided langevin

> Default: 0

> 1: Perform SGLD Method

> 2: Perform SGLDfp method

> 3: Perform SGLDg/SGMDg method

**tsgavg**: Defines the local averaging time for the guiding force calculation in picoseconds

> Default: 0.2

**sgft**: Defines the strength of the guiding effect

> when *gamma_ln*>0:  1.0

> when *gamma_ln*=0: 0.2

> when *isgld=* 1 or 2 and : *tempsg* overrides *sgft*

**tempsg**: Target guiding temperature (Kelvin)





### Pressure Regulation Parameters

**ntp**: Defines the constant pressure dynamics

> 0: (default) No pressure scaling

> 1: Isotropic postion scaling

> 2: ansiotropic pressure scaling

> 3: semiisotropic pressure scaling

**barostat**: Defines which barostat is used to control pressure

> 1: (Default) Berendsen

> 2: Monte Carlo barostat

**pres0**: Defines the reference pressure at which the system is maintained

> Default: 1.0

**taup**: Pressure relaxation time in picoseconds. The recommended value is betwen 1.0 and 50 ps. Larger values may be necessary if the trajectories seeme unstable.

> Default: 1.0

## Potential Function Parameters


#### General Potential Parameters


**ntf**: Force evaluation. When SHAKE is used it is not necessary to calculate the forces for the constrained bonds

**ntb**: Controls whether periodic boundaries are imposed on the system during the calculation of non-bonded interactions. **The variable is determined automatically from *igb* and *ntp* but it can be overridden.**

> 0: (default for *igb* > 0) no periodicity is applied and PME is off

> 1: (default for *igb* = 0 and *ntp* = 0) constant volume

> 2: (default for *ntp* > 0) constant pressure

**cut**: Defines the nonbonded cutoff in Angstroms.

For PME, *cut* is used to limit the direct space sum. 8.0 is a good value.

For *igb*>0, *cut* is used to truncate nonbonded pair interactions that are less than *cut* Angstroms apart. A separate parameter *rgbmax* controls the maximum distance bewteen the atoms pairs that will be considered in carrying out the pairwise summations for calculation of the Born radii.

> For *igb* > 0: (default) 9999.0

> For *igb=0*: (default) 8.0

**ipol**: Flag to use polarizable force field

> 0: (default) do not use polarizable force field

> 1: Use polarizable force field

**ifqnt**: Flag for QM/MM run. If set to 1, the &qmmm namelist must be included

> 0: (default) not a QM/MM run

> 1: This is a QM/MM run

**irism**: Flag for 3D-reference interaction site model (RISM) molecular solvation method.

> 0: (default) do not use RISM

> 1: Use RISM

**ievb**: Flag for using empirical valence bond method to compute energies and force

> 0: (default) do not use empirical valence bond method

> 1: Use the empirical valence bond method

**iamoeba**: Flag for using the amoeba polarizable potential of [Ren and Ponder](https://onlinelibrary.wiley.com/doi/abs/10.1002/jcc.10127). If set to 1, an amoeba namelist must be include.

>0: (default) do not use amoeba polarizable potenital

>1: Use the amoeba polarizable potential




#### Implicit Solvent Parameters

**igb**: Flag for using implicit solvent models

> 0: (default) do not use implicit solvent models

> 1: The [Hawkings, Cramer, Truhlar pairwise generalized Born model](https://pubs.acs.org/doi/full/10.1021/jp961710n)($GB^{HCT}$). Often used for nucleic acid simulations.

> 2: A modified GB model by [A. Onufriev, D. Bashford, and D.A. Case](https://onlinelibrary.wiley.com/doi/abs/10.1002/prot.20033) ($GB^{OBC}$. If you use this, run the LEAP command `set default PBradii mbondi2` to prepare the *prmtop* file.

> 3 or 4: No longer supported

> 5: Same as the *igb=2* but different parameters. Some [tests](https://onlinelibrary.wiley.com/doi/abs/10.1002/jcc.10378) show that *igb=5* agrees more with the Poisson-Boltzmann treatment in calculating the electrostatic part of the solvation free energy.

> 6: No solvent model or more specifically a nonperiodic, vacuum model where the only non-bonded interactions are LJ and Coulomb interactions. Logically equivalent to *igb=0*.

> 7: $GBn$ model described by [ Mongan, Simmerling, McCammon, Case and Onufriev](https://pubs.acs.org/doi/full/10.1021/ct600085e). There is more overhead and is not recommended for fcedureor systems involving nucleic acids.

> 8: Same GB functional form as the GBn (*igb=7*) but contains two sets ofparameters one of which is optimized for proteins and another which is optimized for nucleic acids. **In general, use this for implicit solvent models**.

> 10: Calculate the reaction field and nonbonded interactions using a nubmerical Poisson-Boltzmann solver. Note this is not a generalized Born simulation but rather an alternative continuum solvent model.

**saltcon**: Defines the concentration (M) of 1-1 mobile counterions in solution, using a modified generalized Born theory based on the Debye-Hückel limiting law for ion screening of interactions. Setting *saltcon* to a nonzero value increases computation time.

> Default: 0

**rgbmax**: Defines the maximum distance in Angstroms between atom pairs that will considerd in the pairwise summation involved in calculating the effective Born radii. The recommended value is 26 Angstroms however smaller values from 10-15 Angstroms.

> Default: 25

# Understanding Amber Prmtop file (also called parm or topology files)

Prmtop files are created in TLEAP/XLEAP from the leaprc force field fields and the original molecular structure. They store the simulation parameters such as the number of atoms, the potentials between the atoms, the bond lengths, and the torsion angles that will later be used to calculate the various forces and energies within the system.

A parm file is divided into sections with “%FLAG”s that denote what information each section stores.

You may also see “%FORMAT” comments. These are specific FORTRAN formats that denote the type of variable (i.e. integer, float, char...) being written, the size of the variable, and the number of columns per row in each section.

Here are some examples:

"%FORMAT10i8" denotes that a particular section is storing integers (i) that are represented with up to 8 characters (including characters that are not numerical such s "-" or ".") in groups of 10 (i.e. there will be 10 integers in each row).

"%FORMAT20a4" signifies a section that is storing string characters (a) that are represented with up to 4 characters in groups of 20.

"%FORMAT5E16.8" means that a particular section stores single precision floats with exponents that contain 8 digits in the decimal and that are represented with up to 16 characters (including characters that are not numberical) in groups of 5.



## Section Flags



**%FLAG TITLE**: Stores the title

**%FLAG POINTERS**: Stores the “number of blank” (i.e number of atoms, bonds containing hydrogen, dihedrals…)

**%FLAG ATOM_NAME**: Stores the name of every atom

**%FLAG CHARGE**: Stores the charge of every atom where charges are multiplied by $\sqrt{k_e}$ (18.2223) which is the square root of the electrostatic constant

**%FLAG ATOMIC_NUMBER**: Stores the atomic number of every atom

**%FLAG MASS**: Stores the mass of every atom

**%FLAG ATOM_TYPE_INDEX**: Stores the Lennard Jones index of an atom. Atoms with the same index have the same LJ $\sigma$ and $\epsilon$  parameters.

**%FLAG NUMBER_EXCLUDED_ATOMS**: Stores the number of atoms that need to be excluded from non-bonded calculations for each atom  because it is involved in a bond, angle, or torsion with $i$.

For example, if the first two elements in this section are 4 and 6,  it means that elements 1-4 in the EXCLUDED_ATOMS_LIST section are excluded from atom 1 and elements 6-11 in EXCLUDED_ATOMS_LIST are excluded from atom 2. Each exclusion involving any two atoms is only listed once and assigned to the atom of the lower index (i.e. if atom 1 and 2 are bonded together, atom 2 is on the exclusion list for 1 but not vice versa)

Atoms with no excluded atoms are given a “1” which signifies exclusion of a nonexistent “atom 0”.

**%FLAG NONBONDED_PARM_INDEX**: Stores the pointers for each LJ pair of atoms, i and j, into the LENNARD_JONES_ACOEFF and LENNAR_JONES_BCOEFF arrays.  The pointer is calculated from the ATOM_TYPE_INDEX of  i and j as follows

$${index = \text{NONBONDED_PARM_INDEX}\ [\text{NTYPES}\  \times \ \text{ATOM TYPE INDEX}(i) - 1)\  +\  \text{ATOM TYPE INDEX}(j)]}$$

**%FLAG RESIDUE_LABEL**: Stores the name of each residue

**%FLAG RESIDUE_POINTER**: Stores the index of the first atom in each residue. Note the first atom is at index 1.